In [1]:
# choose your data set:
sim_data_path = 'Traffic_light_sim_only'
real_data_path = 'Traffic_light_real_only'
both_data_path = 'Traffic_light_both'

def get_validation_path(path):
    return path + '/test_images'

In [2]:
# code in this cell based on: https://github.com/keras-team/keras/issues/898

from keras.callbacks import Callback
import keras.backend as K

class LrReducer(Callback):
    def __init__(self, monitor='val_acc', start_accuracy=0.97, patience=5, reduce_rate=0.4, reduce_nb=5, verbose=1):
        super(Callback, self).__init__()
        self.patience = patience
        self.wait = 0
        self.best_score = -1.
        self.reduce_rate = reduce_rate
        self.current_reduce_nb = 0
        self.reduce_nb = reduce_nb
        self.verbose = verbose
        self.start_reduce = False
        self.first_reduce = False
        self.start_accuracy = start_accuracy
        self.monitor = monitor

    def on_epoch_end(self, epoch, logs={}):
        current_score = logs.get(self.monitor)
        if self.start_reduce == False and current_score > self.start_accuracy:
            self.start_reduce = True
            self.first_reduce = True
            self.best_score = current_score
            self.wait = 0
            if self.verbose > 0:
                print('--- LrReducer found starting accuracy: %.4f' % current_score)
                
        if self.first_reduce == False and current_score > self.best_score:
            self.best_score = current_score
            self.wait = 0
            if self.verbose > 0:
                print('--- current best val accuracy: %.4f' % current_score)
        else:
            if self.start_reduce and (self.first_reduce or self.wait >= self.patience):
                self.first_reduce = False
                self.current_reduce_nb += 1
                if self.current_reduce_nb <= self.reduce_nb:
                    lr = K.get_value(self.model.optimizer.lr)
                    K.set_value(self.model.optimizer.lr, lr*self.reduce_rate)
                    if self.verbose > 0:
                        print('--- LrReducer current best val accuracy: %.4f' % current_score)
                        print("--- LrReducer reduced lr to: %.10f" % (lr*self.reduce_rate))
                else:
                    if self.verbose > 0:
                        print("--- Epoch %d: LrReducer early stopping" % (epoch))
                    self.model.stop_training = True
            self.wait += 1

Using TensorFlow backend.


In [3]:
# choose your data set:
train_data_path = both_data_path

validation_data_path = get_validation_path(train_data_path)




from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Convolution2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

batch_size = 64

nb_epoch = 250
nb_classes = 4
image_shape = [64, 64, 3]

model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=image_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# save best model only 
# NOTE: needs validation_data in fit_generator or validation_split in ImageDataGenerator, which isn't available
model_file_name = train_data_path + '_' + 'light_classifier_model.h5'
checkpointer = ModelCheckpoint(monitor='val_acc', filepath=model_file_name, save_best_only=True, verbose=1)

# reduce learning rate when loss doesn't improve much anymore (to pin-point the local minima)
# reduce_lr = ReduceLROnPlateau(monitor='val_acc', epsilon=0.03, factor=0.4, patience=4, min_lr=0.001, verbose=1)
# reduce_lr = LrReducer(monitor='val_acc', start_accuracy=0.96, patience=5, reduce_rate=0.05, reduce_nb=3)


# stop when there was no real improvement anymore (even after reducing the learning rate several times)
# early_stop = EarlyStopping(monitor='val_acc', min_delta=0.01, patience=25, verbose=1)


# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])

datagen = ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, shear_range=0.05, zoom_range=.2,
                             fill_mode='nearest', rescale=1. / 255)

image_train_data_gen = datagen.flow_from_directory(train_data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

image_valid_data_gen = datagen.flow_from_directory(validation_data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

model.fit_generator(image_train_data_gen, nb_epoch=nb_epoch,
                    samples_per_epoch=image_train_data_gen.nb_sample,
                    validation_data=image_valid_data_gen,
                    nb_val_samples=image_valid_data_gen.nb_sample,
                    callbacks=[checkpointer])

# model.save('light_classifier_model.h5')

Found 5067 images belonging to 4 classes.
Found 390 images belonging to 4 classes.
Epoch 1/250
5067/5067 [==============================] - 65s - loss: 1.1553 - acc: 0.5607 - val_loss: 1.1360 - val_acc: 0.4590
Epoch 2/250
5067/5067 [==============================] - 62s - loss: 0.8355 - acc: 0.6481 - val_loss: 1.2279 - val_acc: 0.4923
Epoch 3/250
5067/5067 [==============================] - 62s - loss: 0.7822 - acc: 0.6854 - val_loss: 0.9468 - val_acc: 0.5872
Epoch 4/250
5067/5067 [==============================] - 60s - loss: 0.6516 - acc: 0.7344 - val_loss: 0.9981 - val_acc: 0.5821
Epoch 5/250
5067/5067 [==============================] - 62s - loss: 0.5593 - acc: 0.7750 - val_loss: 0.8321 - val_acc: 0.6205
Epoch 6/250
5067/5067 [==============================] - 61s - loss: 0.5182 - acc: 0.7993 - val_loss: 0.7680 - val_acc: 0.6538
Epoch 7/250
5067/5067 [==============================] - 60s - loss: 0.6540 - acc: 0.7638 - val_loss: 0.8296 - val_acc: 0.6410
Epoch 8/250
5067/5067 [=====

5067/5067 [==============================] - 60s - loss: 0.2308 - acc: 0.8946 - val_loss: 0.4975 - val_acc: 0.7897
Epoch 31/250
5067/5067 [==============================] - 60s - loss: 0.2143 - acc: 0.9074 - val_loss: 0.4688 - val_acc: 0.7795
Epoch 32/250
5067/5067 [==============================] - 60s - loss: 0.2183 - acc: 0.9059 - val_loss: 0.4932 - val_acc: 0.8103
Epoch 33/250
5067/5067 [==============================] - 60s - loss: 0.2239 - acc: 0.9104 - val_loss: 0.3184 - val_acc: 0.8538
Epoch 34/250
5067/5067 [==============================] - 60s - loss: 0.2208 - acc: 0.9080 - val_loss: 0.8737 - val_acc: 0.7308
Epoch 35/250
5067/5067 [==============================] - 61s - loss: 0.2286 - acc: 0.9086 - val_loss: 0.3670 - val_acc: 0.8462
Epoch 36/250
5067/5067 [==============================] - 62s - loss: 0.2086 - acc: 0.9167 - val_loss: 0.3008 - val_acc: 0.8897
Epoch 37/250
5067/5067 [==============================] - 60s - loss: 0.1987 - acc: 0.9199 - val_loss: 0.2860 - val_a

5067/5067 [==============================] - 60s - loss: 0.1155 - acc: 0.9518 - val_loss: 0.2052 - val_acc: 0.9205
Epoch 63/250
5067/5067 [==============================] - 62s - loss: 0.1193 - acc: 0.9536 - val_loss: 0.1874 - val_acc: 0.9410
Epoch 64/250
5067/5067 [==============================] - 62s - loss: 0.1167 - acc: 0.9491 - val_loss: 0.1347 - val_acc: 0.9487
Epoch 65/250
5067/5067 [==============================] - 60s - loss: 0.1096 - acc: 0.9552 - val_loss: 0.1863 - val_acc: 0.9128
Epoch 66/250
5067/5067 [==============================] - 60s - loss: 0.1086 - acc: 0.9576 - val_loss: 0.1648 - val_acc: 0.9077
Epoch 67/250
5067/5067 [==============================] - 60s - loss: 0.1917 - acc: 0.9278 - val_loss: 0.2390 - val_acc: 0.8821
Epoch 68/250
5067/5067 [==============================] - 60s - loss: 0.1404 - acc: 0.9440 - val_loss: 0.1909 - val_acc: 0.9231
Epoch 69/250
5067/5067 [==============================] - 60s - loss: 0.1312 - acc: 0.9489 - val_loss: 0.3248 - val_a

5067/5067 [==============================] - 61s - loss: 0.0987 - acc: 0.9633 - val_loss: 0.1332 - val_acc: 0.9410
Epoch 95/250
5067/5067 [==============================] - 60s - loss: 0.0812 - acc: 0.9688 - val_loss: 0.1272 - val_acc: 0.9564
Epoch 96/250
5067/5067 [==============================] - 61s - loss: 0.1120 - acc: 0.9611 - val_loss: 0.1393 - val_acc: 0.9538
Epoch 97/250
5067/5067 [==============================] - 61s - loss: 0.1099 - acc: 0.9621 - val_loss: 0.1803 - val_acc: 0.9538
Epoch 98/250
5067/5067 [==============================] - 60s - loss: 0.1010 - acc: 0.9619 - val_loss: 0.1948 - val_acc: 0.9231
Epoch 99/250
5067/5067 [==============================] - 60s - loss: 0.0954 - acc: 0.9653 - val_loss: 0.1226 - val_acc: 0.9513
Epoch 100/250
5067/5067 [==============================] - 60s - loss: 0.0981 - acc: 0.9657 - val_loss: 0.1259 - val_acc: 0.9590
Epoch 101/250
5067/5067 [==============================] - 60s - loss: 0.0880 - acc: 0.9680 - val_loss: 0.1350 - val

5067/5067 [==============================] - 61s - loss: 0.0782 - acc: 0.9732 - val_loss: 0.1408 - val_acc: 0.9513
Epoch 127/250
5067/5067 [==============================] - 60s - loss: 0.0947 - acc: 0.9692 - val_loss: 0.2780 - val_acc: 0.9000
Epoch 128/250
5067/5067 [==============================] - 62s - loss: 0.0870 - acc: 0.9659 - val_loss: 0.0791 - val_acc: 0.9846
Epoch 129/250
5067/5067 [==============================] - 60s - loss: 0.0851 - acc: 0.9706 - val_loss: 0.1181 - val_acc: 0.9590
Epoch 130/250
5067/5067 [==============================] - 60s - loss: 0.0984 - acc: 0.9674 - val_loss: 0.1369 - val_acc: 0.9538
Epoch 131/250
5067/5067 [==============================] - 60s - loss: 0.0997 - acc: 0.9627 - val_loss: 0.1101 - val_acc: 0.9513
Epoch 132/250
5067/5067 [==============================] - 60s - loss: 0.0980 - acc: 0.9649 - val_loss: 0.1067 - val_acc: 0.9513
Epoch 133/250
5067/5067 [==============================] - 60s - loss: 0.0813 - acc: 0.9688 - val_loss: 0.0727 

5067/5067 [==============================] - 60s - loss: 0.0860 - acc: 0.9700 - val_loss: 0.1191 - val_acc: 0.9487
Epoch 159/250
5067/5067 [==============================] - 60s - loss: 0.0789 - acc: 0.9728 - val_loss: 0.1407 - val_acc: 0.9538
Epoch 160/250
5067/5067 [==============================] - 60s - loss: 0.0630 - acc: 0.9763 - val_loss: 0.1040 - val_acc: 0.9769
Epoch 161/250
5067/5067 [==============================] - 60s - loss: 0.0657 - acc: 0.9745 - val_loss: 0.0812 - val_acc: 0.9564
Epoch 162/250
5067/5067 [==============================] - 60s - loss: 0.0795 - acc: 0.9718 - val_loss: 0.1303 - val_acc: 0.9513
Epoch 163/250
5067/5067 [==============================] - 60s - loss: 0.0831 - acc: 0.9749 - val_loss: 0.0615 - val_acc: 0.9769
Epoch 164/250
5067/5067 [==============================] - 60s - loss: 0.0926 - acc: 0.9682 - val_loss: 0.0752 - val_acc: 0.9744
Epoch 165/250
5067/5067 [==============================] - 60s - loss: 0.0935 - acc: 0.9734 - val_loss: 0.0964 

5067/5067 [==============================] - 60s - loss: 0.0751 - acc: 0.9765 - val_loss: 0.0901 - val_acc: 0.9744
Epoch 192/250
5067/5067 [==============================] - 60s - loss: 0.0607 - acc: 0.9795 - val_loss: 0.0481 - val_acc: 0.9872
Epoch 193/250
5067/5067 [==============================] - 60s - loss: 0.0676 - acc: 0.9747 - val_loss: 0.1181 - val_acc: 0.9538
Epoch 194/250
5067/5067 [==============================] - 60s - loss: 0.0727 - acc: 0.9759 - val_loss: 0.1337 - val_acc: 0.9615
Epoch 195/250
5067/5067 [==============================] - 60s - loss: 0.0627 - acc: 0.9779 - val_loss: 0.0820 - val_acc: 0.9718
Epoch 196/250
5067/5067 [==============================] - 60s - loss: 0.0798 - acc: 0.9759 - val_loss: 0.1409 - val_acc: 0.9667
Epoch 197/250
5067/5067 [==============================] - 60s - loss: 0.0627 - acc: 0.9775 - val_loss: 0.0870 - val_acc: 0.9769
Epoch 198/250
5067/5067 [==============================] - 60s - loss: 0.0655 - acc: 0.9783 - val_loss: 0.0661 

5067/5067 [==============================] - 61s - loss: 0.0751 - acc: 0.9745 - val_loss: 0.0901 - val_acc: 0.9641
Epoch 225/250
5067/5067 [==============================] - 61s - loss: 0.0856 - acc: 0.9738 - val_loss: 0.0623 - val_acc: 0.9769
Epoch 226/250
5067/5067 [==============================] - 60s - loss: 0.0862 - acc: 0.9720 - val_loss: 0.1017 - val_acc: 0.9692
Epoch 227/250
5067/5067 [==============================] - 61s - loss: 0.0792 - acc: 0.9732 - val_loss: 0.0831 - val_acc: 0.9590
Epoch 228/250
5067/5067 [==============================] - 60s - loss: 0.0871 - acc: 0.9716 - val_loss: 0.0552 - val_acc: 0.9795
Epoch 229/250
5067/5067 [==============================] - 60s - loss: 0.0802 - acc: 0.9718 - val_loss: 0.0664 - val_acc: 0.9744
Epoch 230/250
5067/5067 [==============================] - 60s - loss: 0.0620 - acc: 0.9791 - val_loss: 0.0665 - val_acc: 0.9821
Epoch 231/250
5067/5067 [==============================] - 60s - loss: 0.0687 - acc: 0.9759 - val_loss: 0.0485 

In [4]:
model.load_weights(filepath=model_file_name)

datagen = ImageDataGenerator(fill_mode='nearest', rescale=1. / 255)  # no randomization

data_path = sim_data_path
train_data_gen = datagen.flow_from_directory(data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])
valid_data_gen = datagen.flow_from_directory(get_validation_path(data_path), target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

print("Simulation Training loss and accuracy: ", model.evaluate_generator(train_data_gen, train_data_gen.nb_sample))
print("Simulation Validation loss and accuracy: ", model.evaluate_generator(valid_data_gen, valid_data_gen.nb_sample))


data_path = real_data_path
train_data_gen = datagen.flow_from_directory(data_path, target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])
valid_data_gen = datagen.flow_from_directory(get_validation_path(data_path), target_size=(64, 64), batch_size=batch_size,
                                             classes=['red', 'yellow', 'green', 'none'])

print("RealWorld Training loss and accuracy: ", model.evaluate_generator(train_data_gen, train_data_gen.nb_sample))
print("RealWorld Validation loss and accuracy: ", model.evaluate_generator(valid_data_gen, valid_data_gen.nb_sample))

Found 2216 images belonging to 4 classes.
Found 200 images belonging to 4 classes.
('Simulation Training loss and accuracy: ', [0.011899526674052708, 0.99548736462093868])
('Simulation Validation loss and accuracy: ', [0.020342193030082854, 0.995])
Found 2851 images belonging to 4 classes.
Found 190 images belonging to 4 classes.
('RealWorld Training loss and accuracy: ', [0.012780425150531483, 0.99754472115047355])
('RealWorld Validation loss and accuracy: ', [0.057813581902729838, 0.98947368922986478])


In [5]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 64, 64, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 64, 64, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 62, 62, 32)    9248        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 62, 62, 32)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________